In [1]:
#Pip install selenium and webdriver_manager if not already done

import selenium
from selenium import webdriver
import time
import requests
import os
from PIL import Image
import io
import hashlib
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
from bs4 import BeautifulSoup

In [2]:
# Put the path for your ChromeDriver here
DRIVER_PATH = "ChromeDriverManager().install()"

In [3]:
def fetch_image_urls(query:str, no_of_pages_of_60:int, wd:webdriver, sleep_between_interactions:int=1):
    
    q = query
    
    wd.get(f"https://www.google.co.uk/search?q={q}&hl=en&psb=1&tbs=vw:d&tbm=shop&ei=cn_-Y9DnFt_87_UP2d6-6Ac&start=0&sa=N&ved=0ahUKEwjQ6Luhobn9AhVf_rsIHVmvD30Q8tMDCI4R&biw=1470&bih=733&dpr=2")
    wd.find_element(By.XPATH, "//button[contains(@class, 'VfPpkd')]").click()
    
    image_urls = []
    for n in range(no_of_pages_of_60):
        search_url =f"https://www.google.co.uk/search?q={q}&hl=en&psb=1&tbs=vw:d&tbm=shop&ei=cn_-Y9DnFt_87_UP2d6-6Ac&start={n*60}&sa=N&ved=0ahUKEwjQ6Luhobn9AhVf_rsIHVmvD30Q8tMDCI4R&biw=1470&bih=733&dpr=2"
        wd.get(search_url)

    #get all image thumbnail results
        print('Processing: ')
        for j in range(1,61):
            try:
                thumbnail_xpath = f"/html/body/div[7]/div/div[4]/div[3]/div/div[3]/div[2]/div[2]/div/div[{j}]/div[1]/div[2]/div[1]/div[1]/div/div/a/div/div/img"
                
                thumbnail_results = wd.find_element(By.XPATH, thumbnail_xpath).get_attribute('src')
                image_urls.append(thumbnail_results)
                print(f"{j}/60 ", end="")

            except Exception:
                continue
    
        image_count = len(image_urls)
        print(f"image count: {image_count} after page {n+1}")
        
    return set(image_urls)


In [4]:
def persist_image(folder_path:str,file_name:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        folder_path = os.path.join(folder_path,file_name)
        if os.path.exists(folder_path):
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        else:
            os.mkdir(folder_path)
            file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [ ]:
#change your set of queries here, eg. carrots
query = "carrots"
#set number of pages to scrape -- google shopping search returns 60 items per page
no_of_pages_of_60_to_scrape = 2
#images_path_folder
images_path = '/Users/username/desktop/download_images'

wd = webdriver.Chrome(executable_path=DRIVER_PATH)
links = fetch_image_urls(query, no_of_pages_of_60_to_scrape ,wd)
    
for i in links:
    persist_image(images_path,query,i)